In [49]:
import os, sys, glob
import numpy as np
from matplotlib import pyplot as plt
import h5py
import pandas as pd

# Kareems code

For each flavor: Sum over I3MCWeightDict.weight where QuesoL4_Bool==1 and add all 6 flavor rates

This converter was used: https://github.com/icecube/wg-oscillations-fridge/blob/75fd22ff047f93023617bf794ff237bbcdd50033/processing/samples/icu/old/processing/icu_i3_to_hdf5.py

This corresponds to this file /data/sim/IceCubeUpgrade/pisa/level4_queso_v00/ICU_pisa_genie_028_level4_queso_v00.hdf5

In [68]:
def get_pds_aeff_upgrade(flavour):
    result=pd.DataFrame(np.array([
        GrecoQuesoAeff[flavour]['MCInIcePrimary.energy'][:][:],
    GrecoQuesoAeff[flavour]['MCInIcePrimary.dir.coszen'][:][:],
    GrecoQuesoAeff[flavour]['weighted_aeff'][:][:],
    GrecoQuesoAeff[flavour]['QuesoL4_Bool'][:][:],
    GrecoQuesoAeff[flavour]['I3MCWeightDict.OneWeight'][:][:],
    GrecoQuesoAeff[flavour]['I3GenieInfo.n_flux_events'][:][:],
    GrecoQuesoAeff[flavour]["I3MCWeightDict.weight"][:][:]
    # GrecoQuesoAeff[flavour]['I3GenieInfo.power_law_index'][:][:]
    ]).T,
    columns=['MCInIcePrimaryEnergy','MCInIcePrimaryCosZenith','WeightedAeff','QuesoL4Bool',"I3MCWeight","NumEvents"
            # "PowerLawIndex"
            , "I3WeightWithModel"
            ])
    return result

ICUPGRADE_AEFF_HDF5_FILE="/cvmfs/icecube.opensciencegrid.org/users/kareem/Upgrade/effective_areas/greco_queso/ICU_pisa_genie_028_level4_queso_v00.hdf5"
#ICUPGRADE_AEFF_HDF5_FILE="/data/sim/IceCubeUpgrade/pisa/level4_queso_v00/ICU_pisa_genie_028_level4_queso_v00.hdf5" #he seems to have copied this file

GrecoQuesoAeff=h5py.File(ICUPGRADE_AEFF_HDF5_FILE)

with h5py.File(ICUPGRADE_AEFF_HDF5_FILE, "r") as file:
    keys = list(file.keys())
    print("Available keys:", keys)

Available keys: ['nue_cc', 'nue_nc', 'nuebar_cc', 'nuebar_nc', 'numu_cc', 'numu_nc', 'numubar_cc', 'numubar_nc', 'nutau_cc', 'nutau_nc', 'nutaubar_cc', 'nutaubar_nc']


In [69]:
#create dataframe from .hdf5 file
nue_cc_df=get_pds_aeff_upgrade('nue_cc');
nue_nc_df=get_pds_aeff_upgrade('nue_nc');
numu_cc_df=get_pds_aeff_upgrade('numu_cc')
numu_nc_df=get_pds_aeff_upgrade('numu_nc');
nutau_cc_df=nutau_cc_df=get_pds_aeff_upgrade('nutau_cc');
nutau_nc_df=get_pds_aeff_upgrade('nutau_nc');

#calculate rates
rate_nueCC=np.sum((nue_cc_df[nue_cc_df["QuesoL4Bool"]==1])["I3WeightWithModel"])
rate_nueNC=np.sum((nue_nc_df[nue_nc_df["QuesoL4Bool"]==1])["I3WeightWithModel"])
rate_numuCC=np.sum((numu_cc_df[numu_cc_df["QuesoL4Bool"]==1])["I3WeightWithModel"])
rate_numuNC=np.sum((numu_nc_df[numu_nc_df["QuesoL4Bool"]==1])["I3WeightWithModel"])
rate_nutauCC=np.sum((nutau_cc_df[nutau_cc_df["QuesoL4Bool"]==1])["I3WeightWithModel"])
rate_nutauNC=np.sum((nutau_nc_df[nutau_nc_df["QuesoL4Bool"]==1])["I3WeightWithModel"])

#print results
rate_all=[rate_nueCC,rate_nueNC,rate_numuCC,rate_numuNC,rate_nutauCC,rate_nutauNC]
for r in rate_all:
    var_name = [name for name, value in locals().items() if value is r][0]
    print(f"{var_name}:\t{r*1000:.2f} mHz")
print("Background rate for all flavors: %1.2f mHz"%(sum(rate_all) * 1000))

rate_nueCC:	1.35 mHz
rate_nueNC:	0.12 mHz
rate_numuCC:	2.65 mHz
rate_numuNC:	0.34 mHz
rate_nutauCC:	0.12 mHz
rate_nutauNC:	0.10 mHz
Background rate for all flavors: 4.68 mHz


# Our code

Queso file were prior converted vom .i3 to .npy using https://github.com/mjlarson/nusources_dataset_converters/tree/main

In there:
- QuesoL4_Bool is applied (line 76)
- mcweightdict = I3MCWeightDict (line 86)
- atmo_weight = mcweightdict['weight'] (line 186)

Differences to Kareem:
- For us: QuesoL3_Bool and QuesoL3_Vars_cleaned_num_hits_fid_vol >= 7 applied as well (line 75 & 77). Applying these in Kareems code does not change much.
- Different dataformat: He is using /data/sim/IceCubeUpgrade/pisa/level4_queso_v00/ICU_pisa_genie_028_level4_queso_v00.hdf5 and we are using /data/sim/IceCubeUpgrade/genie/level4_queso/...
- He does not use anti particles i.e. nuebar_cc

In [74]:
queso_e_only = np.load("/home/mdittmer/Basics/ConvertedQuesoFiles/upgrade_queso_120028.npy")
queso_e_only_CC = queso_e_only[queso_e_only['interaction']==1]
queso_e_only_NC = queso_e_only[queso_e_only['interaction']==2]

queso_ebar_only = np.load("/home/mdittmer/Basics/ConvertedQuesoFiles/upgrade_queso_121028.npy")
queso_ebar_only_CC = queso_ebar_only[queso_ebar_only['interaction']==1]
queso_ebar_only_NC = queso_ebar_only[queso_ebar_only['interaction']==2]

queso_mu_only = np.load("/home/mdittmer/Basics/ConvertedQuesoFiles/upgrade_queso_140028.npy")
queso_mu_only_CC = queso_mu_only[queso_mu_only['interaction']==1]
queso_mu_only_NC = queso_mu_only[queso_mu_only['interaction']==2]

queso_mubar_only = np.load("/home/mdittmer/Basics/ConvertedQuesoFiles/upgrade_queso_141028.npy")
queso_mubar_only_CC = queso_mubar_only[queso_mubar_only['interaction']==1]
queso_mubar_only_NC = queso_mubar_only[queso_mubar_only['interaction']==2]

queso_tau_only = np.load("/home/mdittmer/Basics/ConvertedQuesoFiles/upgrade_queso_160028.npy")
queso_tau_only_CC = queso_tau_only[queso_tau_only['interaction']==1]
queso_tau_only_NC = queso_tau_only[queso_tau_only['interaction']==2]

queso_taubar_only = np.load("/home/mdittmer/Basics/ConvertedQuesoFiles/upgrade_queso_161028.npy")
queso_taubar_only_CC = queso_taubar_only[queso_taubar_only['interaction']==1]
queso_taubar_only_NC = queso_taubar_only[queso_taubar_only['interaction']==2]

#### Let us not use anti neutrinos as above and recreate the same scenario as Kareem.
We manually divide by the number of files in /data/sim/IceCubeUpgrade/genie/level4_queso/XXX/ (400 for $\bar{\nu}_\tau$, 500 for $\bar{\nu}_\mu$, 1000 for the rest). 

We modified the converter script later on to do that directly (below)

In [85]:
NrFiles=1000
rate_nueCC=np.sum(queso_e_only_CC['atmo_weight']/NrFiles)
rate_nueNC=np.sum(queso_e_only_NC['atmo_weight']/NrFiles)
rate_numuCC=np.sum(queso_mu_only_CC['atmo_weight']/NrFiles)
rate_numuNC=np.sum(queso_mu_only_NC['atmo_weight']/NrFiles)
rate_nutauCC=np.sum(queso_tau_only_CC['atmo_weight']/NrFiles)
rate_nutauNC=np.sum(queso_tau_only_NC['atmo_weight']/NrFiles)

rate_all=[rate_nueCC,rate_nueNC,rate_numuCC,rate_numuNC,rate_nutauCC,rate_nutauNC]
for r in rate_all:
    var_name = [name for name, value in locals().items() if value is r][0]
    print(f"{var_name}:\t{r*1000:.2f} mHz")
print("Background rate for all flavors: %1.2f mHz"%(sum(rate_all) * 1000))

rate_nueCC:	11.27 mHz
rate_nueNC:	1.00 mHz
rate_numuCC:	22.32 mHz
rate_numuNC:	2.89 mHz
rate_nutauCC:	1.60 mHz
rate_nutauNC:	1.28 mHz
Background rate for all flavors: 40.36 mHz


#### We land at 40.36 mHz as our final result and 24.38 mHz for anti neutrinos.
Did you ever calculate the background rate? We don't know what to expect.

#### Modiying the converter script and just test $\nu_\mu^{CC}$: 
- weight_2 = mcweightdict['weight']/(nevents*args.nfiles). This is just wrong and desperate.
- weight_3 = mcweightdict['weight']/(args.nfiles). This yields the same result as above which we thought is correct in principle.

In [87]:
queso_mu_only_test = np.load("/data/user/bschlueter/software/nusources_dataset_converters/datasets/upgrade_queso_140028_test.npy")
queso_mu_only_CC_test = queso_mu_only_test[queso_mu_only_test['interaction']==1]
queso_mu_only_NC_test = queso_mu_only_test[queso_mu_only_test['interaction']==2]

#M1 = sum(queso_mu_only_CC_test['atmo_weight']) # /1000 nr files * 1000 for mHz
M2 = sum(queso_mu_only_CC_test['weight_2'])
M3 = sum(queso_mu_only_CC_test['weight3'])

#print("Background rate for rate_numuCC using just 'atmo_weight': %1.3f mHz"%(M1))
print("Background rate for rate_numuCC using  atmo_weight/(nevents*args.nfiles): %1.2f mHz"%(M2 * 1000))
print("Background rate for rate_numuCC using  atmo_weight/(nevents): \t \t %1.2f mHz"%(M2 * NrFiles * 1000))
print("Background rate for rate_numuCC using  atmo_weight/args.nfiles: \t %1.2f mHz (same as above)"%(M3 * 1000))
print("Corresponding value from Kareem: 2.65 mHz")

Background rate for rate_numuCC using  atmo_weight/(nevents*args.nfiles): 0.00 mHz
Background rate for rate_numuCC using  atmo_weight/(nevents): 	 	 0.52 mHz
Background rate for rate_numuCC using  atmo_weight/args.nfiles: 	 22.32 mHz (same as above)
Corresponding value from Kareem: 2.65 mHz
